<h1><center>Лабораторная работа № 5</center></h1>
<h3><center>"Нейронные сети"</center></h3>

Необходимо решить задачу распознавания рукописных цифр (датасет MNIST). Задача решается в рамках платформы онлайн-конкурсов по машинному обучению Kaggle. Ссылка на задание.

**1\. Провести предподготовку данных**

В задаче каждая фотография рукописной цифры задана в виде строки из 784 (28x28) значений градации серого для каждого пикселя. Градация от 0 до 255 (0 - белый, 255 - черный). Необходимо отнормировать значения для каждой картинки и получить train и validation датасеты.

**2\. Обучить логистическую регрессиию на scikit-learn**

Здесь нужно получить модель логистической регресии и вычислить точность предсказания на валидационном датасете. Точность должна быть ~91%.

**3\. Создать модель многослойной нейронной сети в keras**

Используя библиотеку для глубокого обучения keras.io необходимо создать нейронную сеть из нескольких слоев. Изучить возможности библиотеки, уметь отвечать на вопросы про Dense слои, методы активации, размерности входных/выходных матриц, метод compile и fit. Провести сравнение с логистической регрессией.

**4\. Построить графики обучения и сделать несколько архитектурных вариаций сети**

Собрать историю обучения сети (см. выход функции fit) и построить график обучения (уменьшения loss на каждой итерации). Сделать несколько вариаций архитектуры сети

**5\. Провести эксперименты со своими рукописными цифрами**

В любом графическом редакторе нарисовать набор из 10-20 цифр необходимого размера (28х28), сохранить в папке. Реализовать функцию, которая считывает все файлы из папки и преобразует в вектор. Прогнать нарисованные цифры через полученную на предыдущем этапе лучшую нейронную сеть и подсчитать % ошибок.

### Решение:

# Импортирование модулей

In [187]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from pathlib import Path
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.preprocessing import image

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

####  Подключаем встроенное построение графиков при помощи jupyter notebook и устанавливаем seed для повторяемости результатов:

In [2]:
%matplotlib inline
np.random.seed(42)

# Константы

In [30]:
COMPETITION_FOLDER_PATH = r'C:\Users\User\kaggle\DigitRecognizer'
DATASET_TRAIN_FILE_NAME = 'train.csv'
DATASET_TEST_FILE_NAME = 'test.csv'
SUBMISSION_FILE_NAME = 'recognizer_submission.csv'
KERAS_SEQUENTIAL_MODEL_FILE_NAME = 'my_model.h5'

PD_TARGET_VARIABLE_COLUMN_NAME = 'label'

MATPLOTLIB_PLOT_STYLE = 'ggplot'

SKLEARN_TEST_SPLIT_TEST_SIZE = 0.2
SKLEARN_TEST_SPLIT_RANDOM_STATE = 42

CLASSES_TOTAL_NUMBER = 10
KERAS_SEQUENTIAL_BATCH_SIZE = 20 # in each iteration, we consider 128 training examples at once
KERAS_SEQUENTIAL_EPOCHS_NUMBER = 25 # we iterate twenty times over the entire training set
HIDDEN_SEQUENTIAL_LAYER_SIZE = 800
KERAS_SEQUENTIAL_INPUT_DIM = 784
KERAS_SEQUENTIAL_VALIDATION_SPLIT = 0.1

# Настройка стиля графиков

In [4]:
plt.style.use(MATPLOTLIB_PLOT_STYLE)

# Загрузка данных

In [5]:
data_folder = Path(COMPETITION_FOLDER_PATH)

file_to_open = data_folder / DATASET_TRAIN_FILE_NAME
df = pd.read_csv(file_to_open, index_col=None)

file_to_open = data_folder / DATASET_TEST_FILE_NAME
df_test = pd.read_csv(file_to_open, index_col=None)

In [6]:
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Выделим столбцы, соответствующие входным признакам (матрица X), а отдельно – выделенный признак (вектор y):

In [7]:
X = df.drop((PD_TARGET_VARIABLE_COLUMN_NAME), axis=1).values
X_final_test = df_test.values
y = df[PD_TARGET_VARIABLE_COLUMN_NAME].values

In [8]:
X.shape

(42000, 784)

In [9]:
X_final_test.shape

(28000, 784)

In [10]:
y.shape

(42000,)

### Нормализируем данные:

In [13]:
X = X.astype('float32')
X_final_test = X_final_test.astype('float32')
X /= 255
X /= 255

### Преобразуем метки в категории:

In [16]:
y = np_utils.to_categorical(y, CLASSES_TOTAL_NUMBER)

### Разобьём данные на обучающую/тестовую выборки в отношении
#### $ (1 - $ SKLEARN_TEST_SPLIT_TEST_SIZE$)$  :  SKLEARN_TEST_SPLIT_TEST_SIZE (80% :  20%):

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = SKLEARN_TEST_SPLIT_TEST_SIZE,
                                                    random_state = SKLEARN_TEST_SPLIT_RANDOM_STATE)

# Обучение модели из keras

### Создадим последовательную модель:

In [18]:
model = Sequential()

### Добавим уровни сети:

In [19]:
model.add(Dense(HIDDEN_SEQUENTIAL_LAYER_SIZE, input_dim=KERAS_SEQUENTIAL_INPUT_DIM, activation='relu', 
                kernel_initializer='normal'))
model.add(Dense(HIDDEN_SEQUENTIAL_LAYER_SIZE, activation='relu', kernel_initializer='normal'))
model.add(Dense(CLASSES_TOTAL_NUMBER, activation='softmax', kernel_initializer='normal'))

### Компилируем модель:

In [20]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 800)               628000    
_________________________________________________________________
dense_2 (Dense)              (None, 800)               640800    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                8010      
Total params: 1,276,810
Trainable params: 1,276,810
Non-trainable params: 0
_________________________________________________________________


### Обучаем модель:

In [25]:
%time
seq_model_history = model.fit(X_train, y_train, batch_size=KERAS_SEQUENTIAL_BATCH_SIZE, epochs=KERAS_SEQUENTIAL_EPOCHS_NUMBER, 
                    validation_split=KERAS_SEQUENTIAL_VALIDATION_SPLIT, verbose=2)

Wall time: 0 ns
Train on 30240 samples, validate on 3360 samples
Epoch 1/25
 - 88s - loss: 0.5497 - acc: 0.8289 - val_loss: 0.3194 - val_acc: 0.9098
Epoch 2/25
 - 84s - loss: 0.2679 - acc: 0.9188 - val_loss: 0.2144 - val_acc: 0.9408
Epoch 3/25
 - 84s - loss: 0.1977 - acc: 0.9396 - val_loss: 0.1599 - val_acc: 0.9533
Epoch 4/25
 - 84s - loss: 0.1556 - acc: 0.9508 - val_loss: 0.1500 - val_acc: 0.9548
Epoch 5/25
 - 84s - loss: 0.1203 - acc: 0.9634 - val_loss: 0.1363 - val_acc: 0.9610
Epoch 6/25
 - 84s - loss: 0.0996 - acc: 0.9689 - val_loss: 0.1169 - val_acc: 0.9643
Epoch 7/25
 - 84s - loss: 0.0865 - acc: 0.9728 - val_loss: 0.1235 - val_acc: 0.9622
Epoch 8/25
 - 84s - loss: 0.0735 - acc: 0.9763 - val_loss: 0.0943 - val_acc: 0.9753
Epoch 9/25
 - 88s - loss: 0.0615 - acc: 0.9802 - val_loss: 0.0914 - val_acc: 0.9726
Epoch 10/25
 - 87s - loss: 0.0568 - acc: 0.9818 - val_loss: 0.0910 - val_acc: 0.9735
Epoch 11/25
 - 85s - loss: 0.0462 - acc: 0.9851 - val_loss: 0.1009 - val_acc: 0.9723
Epoch 12/

### Оценим качество обучения сети на тестовых данных:

In [27]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))

Точность работы на тестовых данных: 96.54%


### Сохраним модель в файл:

In [31]:
file_to_save = data_folder / KERAS_SEQUENTIAL_MODEL_FILE_NAME
model.save(file_to_save)

### Предскажем цифры по тестовому датасету, сохраним результаты в файл и загрузим данный файл на kaggle для проверки:

In [32]:
predicted_y = model.predict(X_final_test)

In [35]:
predicted_y.shape

(28000, 10)

In [41]:
df_test_result = pd.DataFrame(index=df_test.index, columns=['Label',], data = predicted_y.argmax(axis=1))
df_test_result.index += 1
df_test_result.index.name = 'ImageId'

In [42]:
df_test_result.head()

,Label
ImageId,
1,2
2,0
3,9
4,0
5,3


In [47]:
submission_file = data_folder / SUBMISSION_FILE_NAME
df_test_result.to_csv(submission_file)

### Построим графики обучения модели:

In [ ]:
plt.plot(seq_model_history.history['acc'])
plt.plot(seq_model_history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(seq_model_history.history['loss'])
plt.plot(seq_model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Распознавание рукописных цифр

In [181]:
def create_num_dataset(num_folder, img_shape=(28, 28), img_count=10, img_format='.png', grayscale=True):
    numbers = np.empty(shape=(0, img_shape[0] * img_shape[1]))
    for i in range(img_count):
        img_name = 'img' + str(i) + img_format
        img_path = num_folder / img_name
        img = image.load_img(img_path, target_size=img_shape, grayscale=grayscale)

        # Преобразуем изображением в массив numpy
        x = image.img_to_array(img)
        x = x.reshape((1, img_shape[0] * img_shape[1]))

        # Инвертируем и нормализуем изображение
        x = 255 - x
        x /= 255
        # x = np.expand_dims(x, axis=0)
        
        numbers = np.append(numbers, x, axis=0)
    return numbers

In [182]:
img_folder = data_folder / 'images'
my_X = create_num_dataset(img_folder)

In [183]:
my_y = np.arange(0, 10, 1).reshape((10, ))
my_y = np_utils.to_categorical(my_y, CLASSES_TOTAL_NUMBER)
my_scores = model.evaluate(my_X, my_y, verbose=0)
print("Точность работы на тестовых данных: %.2f%%" % (my_scores[1]*100))

Точность работы на тестовых данных: 90.00%


In [184]:
prediction = model.predict(my_X)

In [185]:
my_nums_df = pd.DataFrame(columns=['Label',], data = prediction.argmax(axis=1))
my_nums_df.index.name = 'ImageId'

In [186]:
my_nums_df

,Label
ImageId,
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,2
8,8


# Обучение модели из sklearn

In [211]:
logistic_model = LogisticRegression(multi_class='multinomial', penalty='l1', solver='saga')

In [212]:
print(X_train.shape)
print(y_train.shape)

(33600, 784)
(33600, 10)


In [213]:
%time logistic_model.fit(X_train, y_train.argmax(axis=1))

Wall time: 13min 15s


C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l1', random_state=None, solver='saga',
          tol=0.0001, verbose=0, warm_start=False)

In [214]:
logistic_model.score(X_test, y_test.argmax(axis=1))

0.8265476190476191